In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import time

In [54]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to the dataset
data_dir = '/content/drive/MyDrive/JPEGImages'

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Load the dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


In [55]:
# Load a pretrained ResNet model
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Replace the last layer to fit our number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 50)  # Adjusting the final layer for 50 classes
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Train only the last layer


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [60]:
def validate_model():
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    total_correct = 0
    total_images = 0
    with torch.no_grad():  # No gradients needed
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_images += labels.size(0)

    avg_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_images * 100
    print(f'Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')


In [61]:
# Training and validation function
def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        start_time = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if (i + 1) % 20 == 0:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 20:.4f}')
                running_loss = 0.0

        # Timing each epoch
        epoch_time = time.time() - start_time
        print(f'Training time for epoch {epoch + 1}: {epoch_time:.2f} sec')

        # Validate after each epoch
        validate_model()


In [62]:

# Train the model
train_model(num_epochs=10)  # You can adjust the number of epochs


Epoch 1, Batch 20, Loss: 0.0756
Epoch 1, Batch 40, Loss: 0.0849
Epoch 1, Batch 60, Loss: 0.0968
Epoch 1, Batch 80, Loss: 0.0926
Epoch 1, Batch 100, Loss: 0.0925
Epoch 1, Batch 120, Loss: 0.0806
Training time for epoch 1: 39.00 sec
Validation Loss: 0.3080, Validation Accuracy: 90.92%
Epoch 2, Batch 20, Loss: 0.0948
Epoch 2, Batch 40, Loss: 0.0732
Epoch 2, Batch 60, Loss: 0.0884
Epoch 2, Batch 80, Loss: 0.1067
Epoch 2, Batch 100, Loss: 0.0839
Epoch 2, Batch 120, Loss: 0.0837
Training time for epoch 2: 38.95 sec
Validation Loss: 0.2879, Validation Accuracy: 91.69%
Epoch 3, Batch 20, Loss: 0.0767
Epoch 3, Batch 40, Loss: 0.0798
Epoch 3, Batch 60, Loss: 0.0647
Epoch 3, Batch 80, Loss: 0.0606
Epoch 3, Batch 100, Loss: 0.0780
Epoch 3, Batch 120, Loss: 0.0793
Training time for epoch 3: 38.91 sec
Validation Loss: 0.3025, Validation Accuracy: 91.60%
Epoch 4, Batch 20, Loss: 0.0762
Epoch 4, Batch 40, Loss: 0.0743
Epoch 4, Batch 60, Loss: 0.0643
Epoch 4, Batch 80, Loss: 0.0680
Epoch 4, Batch 100, 